In [1]:
import pandas as pd
import numpy as np
import backtrader as bt
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
from datetime import datetime
import tensorflow as tf

/opt/homebrew/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/3878954045.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [20]:
from arch.unitroot import ADF
from arch.unitroot import KPSS
from arch.unitroot import PhillipsPerron
from sklearn.linear_model import LinearRegression

def strat_creation():

  class spread_strat(bt.Strategy):

      params = (
          ('history', 60), #for formulation of cointegrated relationship
          ('rsi_window', 50), #for feature matrix
          ('sma_window', 50),
          ('btc_path', 'btc_model'),
          ('ltc_path', 'ltc_model'),
          ('eth_path', 'eth_model'),
        )

      def log(self, txt, dt=None):
          ''' Logging function for this strategy '''
          if dt is None:
              # Access the datetime index from the current line in the data series
              dt = self.datas[0].datetime.datetime(0)

          # Check if dt is still a float (the internal representation for Backtrader), and convert it if needed
          if isinstance(dt, float):
              # Convert backtrader float date to datetime
              dt = bt.num2date(dt)

          # Format datetime object to string
          dt_str = dt.strftime('%Y-%m-%d %H:%M:%S')
          print('%s, %s' % (dt_str, txt))

      def close_all_positions(self):
        count = 0
        names = ['BTC', 'LTC', 'ETH']
        for data in self.datas:
          position = self.getposition(data)
          if position.size != 0:
              action = 'SELL' if position.size > 0 else 'BUY'
              self.log_trade(action, -1*position.size, data.close[0], names[count])
              self.close(data)
          count+=1
        return

      def log_trade(self, action, size, price, asset):
        self.trade_logs.append({
                'Date': self.datas[0].datetime.datetime(0).strftime('%Y-%m-%d %H:%M:%S'),
                'Action': action,
                'Size': size,
                'Price': price,
                'Asset': asset
            })
        return

      def transform_prices(self, crypto, window):
        if crypto == 'BTC':
          open = pd.Series(self.datas[0].open.get(size=window))
          high = pd.Series(self.datas[0].high.get(size=window))
          low =  pd.Series(self.datas[0].low.get(size=window))
          close = pd.Series(self.datas[0].close.get(size=window))
          close_ret = pd.Series(self.datas[0].close.get(size=window+1))
          ret = close_ret.pct_change().dropna()
          Moving_average = pd.Series(self.data0_sma.array[-window:])
          vol_ret = pd.Series(self.datas[0].close.get(size=101))
          vol_ret = vol_ret.pct_change().dropna()
          volatility = vol_ret.rolling(window=window).std() * np.sqrt(window)
          volatility = volatility[-window:]
          rsi = self.data0_rsi.get(size=window)
          momentum = close - Moving_average
          momentum /= np.array(Moving_average)*np.array(volatility)
          combined_array = np.concatenate((np.array(open), np.array(high), np.array(low), np.array(close), np.array(ret), np.array(volatility), np.array(rsi), np.array(momentum)))
          input_ = combined_array.flatten().reshape(1,-1)
          return input_
        elif crypto == 'LTC':
          open = pd.Series(self.datas[1].open.get(size=window))
          high = pd.Series(self.datas[1].high.get(size=window))
          low =  pd.Series(self.datas[1].low.get(size=window))
          close = pd.Series(self.datas[1].close.get(size=window))
          close_ret = pd.Series(self.datas[0].close.get(size=window+1))
          ret = close_ret.pct_change().dropna()
          Moving_average = pd.Series(self.data0_sma.array[-window:])
          vol_ret = pd.Series(self.datas[1].close.get(size=101))
          vol_ret = vol_ret.pct_change().dropna()
          volatility = vol_ret.rolling(window=window).std() * np.sqrt(window)
          volatility = volatility[-window:]
          rsi = self.data0_rsi.get(size=window)
          momentum = close - Moving_average
          momentum /= np.array(Moving_average)*np.array(volatility)
          combined_array = np.concatenate((np.array(open), np.array(high), np.array(low), np.array(close), np.array(ret), np.array(volatility), np.array(rsi), np.array(momentum)))
          input_ = combined_array.flatten().reshape(1,-1)
          return input_
        elif crypto == 'ETH':
          open = pd.Series(self.datas[2].open.get(size=window))
          high = pd.Series(self.datas[2].high.get(size=window))
          low =  pd.Series(self.datas[2].low.get(size=window))
          close = pd.Series(self.datas[2].close.get(size=window))
          close_ret = pd.Series(self.datas[0].close.get(size=window+1))
          ret = close_ret.pct_change().dropna()
          Moving_average = pd.Series(self.data0_sma.array[-window:])
          vol_ret = pd.Series(self.datas[2].close.get(size=101))
          vol_ret = vol_ret.pct_change().dropna()
          volatility = vol_ret.rolling(window=window).std() * np.sqrt(window)
          volatility = volatility[-window:]
          rsi = self.data0_rsi.get(size=window)
          momentum = close - Moving_average
          momentum /= np.array(Moving_average)*np.array(volatility)
          combined_array = np.concatenate((np.array(open), np.array(high), np.array(low), np.array(close), np.array(ret), np.array(volatility), np.array(rsi), np.array(momentum)))
          input_ = combined_array.flatten().reshape(1,-1)
          return input_

      def clear_trade_vars(self):
        self.cointegrated = False
        self.model_built = False
        self.spread_val = None
        self.upper_bound = None
        self.lower_bound = None
        self.mean_ = None
        self.coefficients = None
        self.order = None
        self.position_opened = False
        self.position_opened_time = None
        self.stop_loss_spread_val = None
        self.position_type = None
        #self.model_date = self.datas[0].datetime.datetime(0)
        return

      def __init__(self):
        self.model_btc = tf.keras.models.load_model('btc_model_Final.keras')
        self.model_ltc = tf.keras.models.load_model('ltc_model_Final.keras')
        self.model_eth = tf.keras.models.load_model('eth_model_Final.keras')

        self.data0 = self.datas[0].close #btc
        self.data1 = self.datas[1].close #ltc
        self.data2 = self.datas[2].close #eth

        self.data0_rsi = bt.indicators.RSI(self.datas[0], period=self.params.rsi_window)
        self.data1_rsi = bt.indicators.RSI(self.datas[1], period=self.params.rsi_window)
        self.data2_rsi = bt.indicators.RSI(self.datas[2], period=self.params.rsi_window)

        self.data0_sma = bt.indicators.SMA(self.datas[0], period=self.params.sma_window)
        self.data1_sma = bt.indicators.SMA(self.datas[1], period=self.params.sma_window)
        self.data2_sma = bt.indicators.SMA(self.datas[2], period=self.params.sma_window)


        #constant for strat
        self.cointegrated = False
        self.model_built = False
        self.spread_val = None
        self.upper_bound = None
        self.lower_bound = None
        self.mean_ = None
        self.sigma = 1.5
        self.coefficients = None
        self.order = None
        self.position_opened = False
        self.position_opened_time = None
        self.stop_loss_spread_val = None
        self.stop_loss_constant = 0.10
        self.position_type = None
        self.model_date = self.datas[0].datetime.datetime(0)
        self.trade_logs = []


      def calc_ml_forecast(self, btc_input, ltc_input, eth_input):
        btc_predict = self.model_btc.predict(btc_input)[0][0]
        ltc_predict = self.model_ltc.predict(ltc_input)[0][0]
        eth_predict = self.model_eth.predict(eth_input)[0][0]

        self.log(f'predictions: btc:{self.data0[0]}, pred:{btc_predict}, ltc:{self.data1[0]}, pred:{ltc_predict}, eth:{self.data2[0]}, pred:{eth_predict}')

        ml_forecast = btc_predict + self.coefficients[1]*ltc_predict + self.coefficients[2]*eth_predict

        return ml_forecast

      #trading during best time
      def next(self):
        current_datetime = self.datas[0].datetime.datetime(0)
        current_hour = current_datetime.hour
        current_minute = current_datetime.minute
        series1 = pd.Series(self.data0.get(size=self.params.history))
        series2 = pd.Series(self.data1.get(size=self.params.history))
        series3 = pd.Series(self.data2.get(size=self.params.history))
        series1_diff = series1.diff().dropna()  # taking the diff of both series
        series2_diff = series2.diff().dropna()
        series3_diff = series3.diff().dropna()
        btc_ret = series1.pct_change()
        btc_ret = btc_ret.dropna()
        ltc_ret = series2.pct_change()
        ltc_ret = btc_ret.dropna()
        eth_ret = series3.pct_change()
        eth_ret = eth_ret.dropna()
        tickers = [series1_diff, series2_diff, series3_diff]
        window = 50
        if len(self.data0) > window*2+1: #check if there is sufficient data for looking back
          input_btc = self.transform_prices('BTC', window)
          input_ltc= self.transform_prices('LTC', window)
          input_eth = self.transform_prices('ETH', window)
          if self.model_built == False:
            ##we build the cointegrated model
            for i in range(len(tickers)): #iterate through series and check for stationarity
              adf = ADF(tickers[i])
              pp = PhillipsPerron(tickers[i])
              kpss = KPSS(tickers[i])
              #print(f'adf: {adf.pvalue}, pp: {pp.pvalue}, kpss:{kpss.pvalue}')
              if adf.pvalue < 0.05 and pp.pvalue < 0.05 and kpss.pvalue > 0.10: #check weather pass all the tests
                self.cointegrated = True
              else:
                self.cointegrated = False

          if self.cointegrated == True and self.model_built == False:
            ##we create the model now
            lin_model = LinearRegression()
            X = np.column_stack((series2.values, series3.values)) #ETH and LTC values
            lin_model.fit(X, series1.values)
            self.coefficients = [1] + list(-1*lin_model.coef_)
            spread = series1 + self.coefficients[1]*series2 + self.coefficients[2]*series3
            self.mean_ = np.mean(spread)
            self.upper_bound = self.mean_ + self.sigma*np.std(spread)
            self.lower_bound = self.mean_ - self.sigma*np.std(spread)
            self.model_built = True
            self.model_date = current_datetime
            self.spread_val = self.data0[0] + self.coefficients[1]*self.data1[0] + self.coefficients[2]*self.data2[0]
            #self.log(f'Lower bound {self.lower_bound}, Upper bound {self.upper_bound}, spread_val: {self.spread_val}, coefs:{self.coefficients}')

          if self.model_built == True:
            self.spread_val = self.data0[0] + self.data1[0]*self.coefficients[1] + self.data2[0]*self.coefficients[2]
            #self.log(f'spread val: {spread_val}')
            if self.spread_val < self.lower_bound or self.spread_val > self.upper_bound: #only enter if spread value is above or below the bounds
              if not self.position:
                #compute ml forecast
                ml_forecast = self.calc_ml_forecast(input_btc, input_ltc, input_eth)
                self.log(f'current spread: {self.spread_val}, Lower bound {self.lower_bound}, Upper bound {self.upper_bound}, forecast pred: {ml_forecast}')
                #we go LONG the spread
                lower_bound_threshold = self.lower_bound * 1.0 if self.lower_bound > 0 else self.lower_bound * 1.0
                upper_bound_threshold = self.upper_bound * 1.0 if self.upper_bound > 0 else self.upper_bound * 1.0
                if self.spread_val < self.lower_bound and ml_forecast > lower_bound_threshold:
                  self.log(f'BUY CREATED, spread val: {self.spread_val}, ml pred: {ml_forecast}')
                  self.buy(self.datas[0], size = 1) #buy BTC
                  self.log_trade('BUY', 1, self.datas[0][0], 'BTC')
                  self.position_opened = True
                  self.position_opened_time = current_datetime
                  for i in range(1, len(self.coefficients)):
                    if self.coefficients[i] > 0:
                      self.buy(self.datas[i], size=round(self.coefficients[i],2))
                      self.log_trade('BUY', self.coefficients[i], self.datas[i][0], 'LTC' if i == 1 else 'ETH')
                    if self.coefficients[i] < 0:
                      self.sell(self.datas[i], size=round(self.coefficients[i],2))
                      self.log_trade('SELL', self.coefficients[i], self.datas[i][0], 'LTC' if i == 1 else 'ETH')
                  self.stop_loss_spread_val = self.spread_val - self.spread_val * self.stop_loss_constant if self.spread_val > 0 else self.spread_val + self.spread_val * self.stop_loss_constant
                  self.position_type = "LONG"
                #we go SHORT the spread
                if self.spread_val > self.upper_bound and ml_forecast < upper_bound_threshold:
                  self.log(f'SELL CREATED, spread val: {self.spread_val}, ml pred: {ml_forecast}')
                  self.sell(self.datas[0], size = 1) #sell BTC
                  self.log_trade('SELL', -1, self.datas[0][0], 'BTC')
                  self.position_opened = True
                  self.position_opened_time = current_datetime
                  for i in range(1, len(self.coefficients)):
                    if self.coefficients[i] > 0:
                      self.sell(self.datas[i], size= round(self.coefficients[i],2))
                      self.log_trade('SELL', -1*self.coefficients[i], self.datas[i][0], 'LTC' if i == 1 else 'ETH')
                    if self.coefficients[i] < 0:
                      self.buy(self.datas[i], size = round(self.coefficients[i],2))
                      self.log_trade('BUY', -1*self.coefficients[i], self.datas[i][0], 'LTC' if i == 1 else 'ETH')
                  self.stop_loss_spread_val = self.spread_val + self.spread_val * self.stop_loss_constant if self.spread_val > 0 else self.spread_val - self.spread_val * self.stop_loss_constant
                  self.position_type = "SHORT"

              #close out and take profits
              elif self.position:
                position_size = self.position.size
                #close out our LONG position
                if position_size > 0 and self.spread_val > self.upper_bound:
                  self.log(f'CLOSE EXISTING long position: {self.spread_val}')
                  self.close_all_positions()
                  self.clear_trade_vars()
                elif position_size < 0 and self.spread_val < self.lower_bound:
                  self.log(f'CLOSE EXISTING short positon: {self.spread_val}')
                  self.close_all_positions()
                  self.clear_trade_vars()
                elif self.position:
                  self.check_stop_loss()

          if self.model_built == True and self.position_opened == False: #only rebuild if not in a position
           if abs(self.spread_val) > abs(1.30*self.upper_bound) or abs(self.spread_val) < abs((1-.30)*self.lower_bound): #if the current spread is to far away from out cointegrated model we rebuild the model
            self.log('REBUILD THE MODEL')
            self.clear_trade_vars()

          days_close = 90
          if self.position_opened and (current_datetime - self.position_opened_time).days >= days_close:
            self.log(f'POSITION CLOSED after {days_close} days at {current_datetime}, spread val: {self.spread_val}')
            self.close_all_positions()
            self.clear_trade_vars()

      def check_stop_loss(self):
        if self.spread_val < self.stop_loss_spread_val and self.position_type == "LONG":
          self.log(f'Long position stopped out: {self.spread_val}')
          self.close_all_positions()
          self.clear_trade_vars()
        elif self.spread_val > self.stop_loss_spread_val and self.position_type == "SHORT":
          self.log(f'Short position stopped out: {self.spread_val}')
          self.close_all_positions()
          #print('in stop')
          self.clear_trade_vars()

        return

      def stop(self):
        # Close all positions at the end of the strategy
        self.log('Closing all positions at the end of the backtest')
        self.close_all_positions()
        self.trade_logs_df = pd.DataFrame(self.trade_logs)
        self.trade_logs_df['value'] = self.trade_logs_df['Size']*self.trade_logs_df['Price']
        print(self.trade_logs_df)

  return spread_strat

In [21]:
#clean up data for
data = pd.read_csv('full_data.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.drop('Unnamed: 0', axis = 1)

btc = data.loc[data['Crypto']=='BTC']
btc = btc.sort_values(by='Date')
btc = btc.reset_index(drop=True)
btc = btc.drop(['Crypto'],axis=1)
btc = btc.set_index('Date')

eth = data.loc[data['Crypto']=='ETH']
eth = eth.sort_values(by='Date')
eth = eth.reset_index(drop=True)
eth = eth.drop(['Crypto'],axis=1)
eth = eth.set_index('Date')

ltc = data.loc[data['Crypto']=='LTC']
ltc = ltc.sort_values(by='Date')
ltc = ltc.reset_index(drop=True)
ltc = ltc.drop(['Crypto'],axis=1)
ltc = ltc.set_index('Date')

#train for optimization only till then
btc = btc.loc[btc.index>=datetime(2022,2,28)]
eth = eth.loc[eth.index>=datetime(2022,2,28)]
ltc = ltc.loc[ltc.index>=datetime(2022,2,28)]

In [22]:
cerebro = bt.Cerebro()
TestStrategy = strat_creation()
cerebro.addstrategy(TestStrategy)
btc_bt = bt.feeds.PandasData(dataname=btc)
cerebro.adddata(btc_bt)
ltc_bt = bt.feeds.PandasData(dataname=ltc)
cerebro.adddata(ltc_bt)
eth_bt = bt.feeds.PandasData(dataname=eth)
cerebro.adddata(eth_bt)
initial_cash = 1000000
cerebro.broker.set_cash(initial_cash)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trade_analyzer")
cerebro.addanalyzer(bt.analyzers.Transactions, _name='transactions')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe_ratio", riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")
cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
cerebro.broker.setcommission(commission=0.0)

results = cerebro.run()
strategy = results[0]


metrics = {
        'method': [],
        'Total Net Profit': [],
        'Gross Profit': [],
        'Gross Loss': [],
        'Percent Profitable': [],
        'Winning Trades': [],
        'Losing Trades': [],
        'Avg. Trade Net Profit': [],
        'Avg. Winning Trade': [],
        'Avg. Losing Trade': [],
        'Ratio Avg. Win:Avg. Loss': [],
        'Largest Winning Trade': [],
        'Largest Losing Trade': [],
        'Max. Consecutive Winning Trades': [],
        'Max. Consecutive Losing Trades': [],
        'Avg. Bars in Total Trades': [],
        'Avg. Bars in Winning Trades': [],
        'Avg. Bars in Losing Trades': [],
        'Max. Drawdown': []
    }
    
# Extract metrics from analyzers
trade_analyzer = strategy.analyzers.trade_analyzer.get_analysis()
sharpe_ratio = strategy.analyzers.sharpe_ratio.get_analysis()
drawdown = strategy.analyzers.drawdown.get_analysis()
returns = strategy.analyzers.returns.get_analysis()
# Calculate custom metrics
total_net_profit = trade_analyzer.won.pnl.total + trade_analyzer.lost.pnl.total
total_gross_profit = trade_analyzer.won.pnl.total
total_gross_loss = trade_analyzer.lost.pnl.total
total_number_trades = trade_analyzer.total.total
percent_profitable = (trade_analyzer.won.total / trade_analyzer.total.total) * 100
winning_trades = trade_analyzer.won.total
losing_trades = trade_analyzer.lost.total
avg_trade_net_profit = total_net_profit / trade_analyzer.total.total
avg_winning_trade = trade_analyzer.won.pnl.average
avg_losing_trade = trade_analyzer.lost.pnl.average
ratio_avg_win_loss = avg_winning_trade/avg_losing_trade
largest_winning_trade = trade_analyzer.won.pnl.max
largest_losing_trade = trade_analyzer.lost.pnl.max
max_consecutive_winning_trades = trade_analyzer.streak.won.longest
max_consecutive_losing_trades = trade_analyzer.streak.lost.longest
avg_bars_in_total_trades = trade_analyzer.len.total
avg_bars_in_winning_trades = trade_analyzer.len.won.total
avg_bars_in_losing_trades = trade_analyzer.len.lost.total
max_drawdown = drawdown.max.moneydown

metrics['method'].append('ML t+5')
metrics['Total Net Profit'].append(total_net_profit)
metrics['Gross Profit'].append(total_gross_profit)
metrics['Gross Loss'].append(total_gross_loss)
metrics['Percent Profitable'].append(percent_profitable)
metrics['Winning Trades'].append(winning_trades)
metrics['Losing Trades'].append(losing_trades)
metrics['Avg. Trade Net Profit'].append(avg_trade_net_profit)
metrics['Avg. Winning Trade'].append(avg_winning_trade)
metrics['Avg. Losing Trade'].append(avg_losing_trade)
metrics['Ratio Avg. Win:Avg. Loss'].append(ratio_avg_win_loss)
metrics['Largest Winning Trade'].append(largest_winning_trade)
metrics['Largest Losing Trade'].append(largest_losing_trade)
metrics['Max. Consecutive Winning Trades'].append(max_consecutive_winning_trades)
metrics['Max. Consecutive Losing Trades'].append(max_consecutive_losing_trades)
metrics['Avg. Bars in Total Trades'].append(avg_bars_in_total_trades)
metrics['Avg. Bars in Winning Trades'].append(avg_bars_in_winning_trades)
metrics['Avg. Bars in Losing Trades'].append(avg_bars_in_losing_trades)
metrics['Max. Drawdown'].append(max_drawdown)

pd.DataFrame.from_dict(metrics).T

/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:181: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series1 = pd.Series(self.data0.get(size=self.params.history))
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:182: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series2 = pd.Series(self.data1.get(size=self.params.history))
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:183: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series3 = pd.Series(self.data2.get(size=self.params.history))
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/11904670

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
2022-06-09 00:00:00, predictions: btc:30139.0, pred:33633.625, ltc:60.09, pred:55.73324966430664, eth:1792.44, pred:1934.0792236328125
2022-06-09 00:00:00, current spread: 15003.004033412906, Lower bound 11758.078174434017, Upper bound 14692.795786759703, forecast pred: 19050.697721289846
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
2022-06-10 00:00:00, predictions: btc:29194.0, pred:33132.96875, ltc:56.86, pred:54.97584915161133, eth:1677.39, pred:1646.4757080078125
2022-06-10 00:00:00, current spread: 14909.087083791279, Lower bound 11758.078174434017, Upper bound 14692.795786759703, forecast pred: 19271.967942851246
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
2022-06-11 00:00:00, predictions: btc:28393.0, pred:32926.63671875, ltc:52.03, 

/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:68: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  volatility = volatility[-window:]
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:86: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  volatility = volatility[-window:]
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:104: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index i

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
2022-06-12 00:00:00, predictions: btc:27383.0, pred:32305.16796875, ltc:50.04, pred:49.35011672973633, eth:1477.97, pred:1606.0675048828125
2022-06-12 00:00:00, current spread: 14807.920383062987, Lower bound 11758.078174434017, Upper bound 14692.795786759703, forecast pred: 19605.87910991401
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
2022-06-14 00:00:00, predictions: btc:21992.0, pred:28378.76171875, ltc:45.03, pred:40.631507873535156, eth:1187.3, pred:1449.7139892578125
2022-06-14 00:00:00, current spread: 10961.933018945885, Lower bound 11758.078174434017, Upper bound 14692.795786759703, forecast pred: 17667.72033414277
2022-06-14 00:00:00, BUY CREATED, spread val: 10961.933018945885, ml pred: 17667.72033414277
2022-06-16 00:00:00, Long position stopped out: 9703.566125003175


/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:68: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  volatility = volatility[-window:]
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:86: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  volatility = volatility[-window:]
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:104: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index i

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
2023-03-23 00:00:00, predictions: btc:28312.0, pred:31939.224609375, ltc:93.17, pred:79.11164855957031, eth:1817.2, pred:1770.7969970703125
2023-03-23 00:00:00, current spread: -5865.478281573352, Lower bound -7623.211679150214, Upper bound -6031.685001403659, forecast pred: -1751.2037701750814
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
2023-03-24 00:00:00, predictions: btc:27632.0, pred:31400.609375, ltc:90.67, pred:82.53229522705078, eth:1750.0, pred:1567.6241455078125
2023-03-24 00:00:00, current spread: -5250.375106280349, Lower bound -7623.211679150214, Upper bound -6031.685001403659, forecast pred: 1988.3720229762948
2023-03-24 00:00:00, REBUILD THE MODEL
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
2023-03-25 00:00:00, predictions: b

/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:68: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  volatility = volatility[-window:]
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:86: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  volatility = volatility[-window:]
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:104: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index i

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
2023-04-04 00:00:00, predictions: btc:28266.0, pred:31845.69140625, ltc:93.21, pred:80.44523620605469, eth:1879.0, pred:1900.3780517578125
2023-04-04 00:00:00, current spread: -9873.218031167442, Lower bound -9829.065193789018, Upper bound -8095.264422495678, forecast pred: -7269.47062629438
2023-04-04 00:00:00, BUY CREATED, spread val: -9873.218031167442, ml pred: -7269.47062629438
2023-04-14 00:00:00, Long position stopped out: -12652.577661488562
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2023-04-15 00:00:00, predictions: btc:30375.0, pred:34209.62109375, ltc:96.27, pred:81.47296905517578, eth:2089.25, pred:2079.7724609375
2023-04-15 00:00:00, current spread: -7978.517548409742, Lower bound -6962.076770326873, Upper bound -4365.4914478288165, forecast pred: -4296.481790838501
2023-04-15 00:00:00, BUY CR

/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:68: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  volatility = volatility[-window:]
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:86: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  volatility = volatility[-window:]
/var/folders/mb/s55_rmmx4gq6kyms8tszjclh0000gn/T/ipykernel_32391/1190467029.py:104: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index i

,0
method,ML t+5
Total Net Profit,-9621.5891
Gross Profit,15437.2449
Gross Loss,-25058.834
Percent Profitable,44.444444
Winning Trades,16
Losing Trades,17
Avg. Trade Net Profit,-267.266364
Avg. Winning Trade,964.827806
Avg. Losing Trade,-1474.049059


In [24]:
df_ml_5 = pd.DataFrame.from_dict(metrics).T
df_ml_5.to_csv('final_writeup/ML_t_5.csv')